In [ ]:
import cv2
import matplotlib.pyplot as plt
import numpy as np

MAX_FEATURES = 500
GOOD_MATCH_PERCENT = 0.15

def alignImages(im1, im2):
  # Convert images to grayscale
  im1Gray = cv2.cvtColor(im1, cv2.COLOR_BGR2GRAY)
  im2Gray = cv2.cvtColor(im2, cv2.COLOR_BGR2GRAY)
  # Detect ORB features and compute descriptors.
  orb = cv2.ORB_create(MAX_FEATURES)
  keypoints1, descriptors1 = orb.detectAndCompute(im1Gray, None)
  keypoints2, descriptors2 = orb.detectAndCompute(im2Gray, None)
  # Match features.
  matcher = cv2.DescriptorMatcher_create(cv2.DESCRIPTOR_MATCHER_BRUTEFORCE_HAMMING)
  matches = matcher.match(descriptors1, descriptors2, None)
  # Sort matches by score
  matches.sort(key=lambda x: x.distance, reverse=False)
  # Remove not so good matches
  numGoodMatches = int(len(matches) * GOOD_MATCH_PERCENT)
  matches = matches[:numGoodMatches]
  # Draw top matches
  imMatches = cv2.drawMatches(im1, keypoints1, im2, keypoints2, matches, None)
  cv2.imwrite("matches.jpg", imMatches)
  # Extract location of good matches
  points1 = np.zeros((len(matches), 2), dtype=np.float32)
  points2 = np.zeros((len(matches), 2), dtype=np.float32)
  for i, match in enumerate(matches):
    points1[i, :] = keypoints1[match.queryIdx].pt
    points2[i, :] = keypoints2[match.trainIdx].pt
  # Find homography
  h, mask = cv2.findHomography(points1, points2, cv2.RANSAC)
  # Use homography
  height, width, channels = im2.shape
  im1Reg = cv2.warpPerspective(im1, h, (width, height))
  return im1Reg, h



refimg = cv2.imread("C:\\Users\\admin\\Desktop\\TY Project\\Gerber reference.png")

testimg = cv2.imread("C:\\Users\\admin\\Desktop\\TY Project\\Gerber test3.png")

refimg =cv2.resize(refimg,(500,500))
testimg = cv2.resize(testimg,(500,500))
cv2.imshow("Reference Image",refimg)
cv2.imshow("Test Image",testimg)
#cv2.imshow("Reference Image",refimg)
#cv2.imshow("Test Image",testimg)
cv2.imwrite("RefImage resized.jpg",refimg)
cv2.imwrite("TestImg resized.jpg",testimg)
imReg, h = alignImages(testimg,refimg)
cv2.imshow("Aligned Image",imReg)
cv2.imwrite("alignedImage.jpg ",imReg)
diffImg = cv2.absdiff(imReg,refimg)
cv2.imshow("Difference image",diffImg)
cv2.imwrite("difffimg.jpg",diffImg)
ret,thresh1 = cv2.threshold(diffImg,140,255,cv2.THRESH_BINARY)
cv2.imshow("Thresholded image",thresh1)
cv2.imwrite("threshimg.jpg",thresh1)
#kernel = np.ones((3,3), np.uint8)
#dilated=cv2.dilate(thresh1,kernel)
#cv2.imshow("dilated image",dilated)
diffImg2=cv2.medianBlur(thresh1,3)
cv2.imshow("Difference image 2",diffImg2)
cv2.imwrite("difffimg2.jpg",diffImg2)
cv2.waitKey(0)


-1